In [1]:
import csv
import pandas as pd
import numpy as np

import os as os
import sys as sys
from collections import defaultdict
import time
from IPython.display import display

import traceback

In [2]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time1 =", current_time)

Current Time1 = 16:09:25


In [3]:
print('Number of arguments:', len(sys.argv), 'arguments.')
print('Argument List:', str(sys.argv))
print('---')
from_cmdLine = sys.argv[-1]
print(from_cmdLine)

print('---', flush=True)


Number of arguments: 3 arguments.
Argument List: ['/Users/alng/miniconda3/envs/leabra/lib/python3.7/site-packages/ipykernel_launcher.py', '-f', '/Users/alng/Library/Jupyter/runtime/kernel-3f778f47-87ba-40f3-b6b4-02d2dd379eaf.json']
---
/Users/alng/Library/Jupyter/runtime/kernel-3f778f47-87ba-40f3-b6b4-02d2dd379eaf.json
---


In [4]:
if from_cmdLine == 'cmd' :
    data_file = sys.argv[-2]
    print('using command line to set data')
    dataDir = data_file + '/'
else :
    data_file = '2022-01-07-16-09-16'
    dataDir = 'data/' + data_file + '/'


In [5]:
dataDir

'data/2022-01-07-16-09-16/'

In [7]:
# Set dimensions of hidden layer
hidden_dimensions = 1

# Set tasks being run
color_task_run = os.path.exists(dataDir + 'schlichting_Base_Params_TaskColorRecall.csv')
face_task_run = os.path.exists(dataDir + 'schlichting_Base_Params_TaskFaceRecall.csv')


if color_task_run and not face_task_run :
    task_run = 'just_color'
elif color_task_run and face_task_run :
    task_run = 'interleaved'
elif not color_task_run and face_task_run :
    task_run = 'just_face'
    
else:
    task_run = 0
    raise ValueError('something went wrong. Task not run properly.')

    


In [10]:
def get_parameter_values():
    if task_run != 'just_face':
        params_file = dataDir + "schlichting_Base_Params_TaskColorRecall.csv"
    else: 
        params_file = dataDir + "schlichting_Base_Params_TaskFaceRecall.csv"
        
    Layer_ThrP_NMPH_dict = {}
    Layer_Drev_dict = {}
    Layer_DThr_dict = {}
    LTD_mult_TaskColorRecall = None
    Layer_OscAmnt_dict = {}
    Layer_Gi_dict = {}
    overlap_dict = {}
    
    parameter_values = {}

    with open(params_file, 'r') as f:
        for line in f.readlines():
            if line[:6] == "Layer:": # get layer name
                layername = line.split(" ")[-1][:-1]

            if line[:5] == "Prjn:": # get layer name
                prjnname = line.split(" ")[-1][:-1]
                
            if "OscAmnt" in line: # get oscAmnt
                Layer_OscAmnt_dict[layername] = float(line.split(" ")[-2])
            if "Layer" in line and "Gi" in line:
                Layer_Gi_dict[layername] = float(line.split(" ")[9])
                
            if "ThrP_NMPH" in line: # get AveL value (only occurs 4 times in csv file)
                Layer_ThrP_NMPH_dict[prjnname] = float(line.split(" ")[-8])
            if "DRev_NMPH" in line: # get AveL value (only occurs 4 times in csv file)
                Layer_Drev_dict[prjnname] = float(line.split(" ")[-20])
            if (LTD_mult_TaskColorRecall == None) and ('LTD_mult' in line):
                LTD_mult_TaskColorRecall = float(line.split(" ")[-2])
            if "DThr_NMPH" in line:
                Layer_DThr_dict[prjnname] = float(line.split(" ")[-23])

        
    Color_to_Hidden_wt_scale = {}
    with open(params_file, 'r') as f:
        for line in f.readlines():
            if line[:5] == "Prjn:": # get layer name
                prjnname = line.split(" ")[-1][:-1]
            if "Rel:" in line: # get AveL value (only occurs 4 times in csv file)
                Color_to_Hidden_wt_scale[prjnname] = float(line.split(" ")[-1])
            if "NumOverlapUnits" in line:
                
                line_split = line.split(" ")
                overlap_dict['numOverlapUnits'] = int(line_split[[i for i, x in enumerate(line_split) if "NumOverlapUnits" in x][0] + 1])
                overlap_dict['numTotalUnits'] = int(line_split[[i for i, x in enumerate(line_split) if "NumTotalUnits" in x][0] + 1])
                overlap_dict['overlapType'] = str(overlap_dict['numOverlapUnits']) + '/' + str(overlap_dict['numTotalUnits'])



                #### OLD VERSION, BEFORE ADDING SPARSE POSSIBILITY. 
#                 overlap_dict['numOverlapUnits'] = int(line.split(" ")[-3])
#                 overlap_dict['numTotalUnits'] = int(line.split(" ")[-1])
#                 overlap_dict['overlapType'] = str(overlap_dict['numOverlapUnits']) + '/' + str(overlap_dict['numTotalUnits'])
        
    parameter_values['ThrP_NMPH'] = Layer_ThrP_NMPH_dict
    parameter_values['DRev_NMPH'] = Layer_Drev_dict
    parameter_values['DThr_NMPH'] = Layer_DThr_dict

    parameter_values['LTD_mult'] = LTD_mult_TaskColorRecall
    parameter_values['Wt_Scale'] = Color_to_Hidden_wt_scale
    parameter_values['OscAmnt'] = Layer_OscAmnt_dict
    parameter_values['Gi'] = Layer_Gi_dict
    
    parameter_values['Num_units_per_layer'] = {
        'Object': 2, 'Face': 6, 'Output': 50
    }
    if hidden_dimensions == 1:
        parameter_values['Num_units_per_layer']['Hidden'] = 50
    elif hidden_dimensions == 2:
        parameter_values['Num_units_per_layer']['Hidden'] = 100

    parameter_values['overlap'] = overlap_dict
    return parameter_values

# def get_parameter_values():
#     params_file = dataDir + "color_diff_Base_Params_TaskColorWOOsc.csv"
#     Color_to_Hidden_wt_scale = {}

#     with open(params_file, 'r') as f:
#         for line in f.readlines():
#             if line[:5] == "Prjn:": # get layer name
#                 prjnname = line.split(" ")[-1][:-1]
#             if "Rel:" in line: # get AveL value (only occurs 4 times in csv file)
#                 Color_to_Hidden_wt_scale[prjnname] = float(line.split(" ")[-1])

#     parameter_values = {}
#     parameter_values['Wt_Scale'] = Color_to_Hidden_wt_scale
#     return parameter_values

print('getting parameter values', flush=True)

parameter_values = get_parameter_values() 

getting parameter values


In [11]:
parameter_values

{'ThrP_NMPH': {'HiddenToCategory': 0.46,
  'HiddenToScene': 0.4,
  'SceneToHidden': 0.4,
  'CategoryToHidden': 0.46,
  'OutputToHidden': 0.4,
  'HiddenToHidden': 0.4,
  'HiddenToOutput': 0.4},
 'DRev_NMPH': {'HiddenToCategory': 0.3,
  'HiddenToScene': 0.23,
  'SceneToHidden': 0.23,
  'CategoryToHidden': 0.3,
  'OutputToHidden': 0.23,
  'HiddenToHidden': 0.23,
  'HiddenToOutput': 0.23},
 'DThr_NMPH': {'HiddenToCategory': 0.2,
  'HiddenToScene': 0.11,
  'SceneToHidden': 0.11,
  'CategoryToHidden': 0.2,
  'OutputToHidden': 0.11,
  'HiddenToHidden': 0.11,
  'HiddenToOutput': 0.11},
 'LTD_mult': 0.11111112,
 'Wt_Scale': {'HiddenToCategory': 1.0,
  'HiddenToScene': 1.0,
  'SceneToHidden': 1.0,
  'CategoryToHidden': 1.0,
  'OutputToHidden': 1.0,
  'HiddenToHidden': 1.0,
  'HiddenToOutput': 1.0},
 'OscAmnt': {'Category': 0.0,
  'Scene': 0.156,
  'Hidden': 0.0623,
  'Output': 0.03},
 'Gi': {'Category': 1.8, 'Scene': 0.9, 'Hidden': 1.0, 'Output': 1.8},
 'Num_units_per_layer': {'Object': 2, 'Face

In [15]:
def get_parameter_values(model_name = "Chanales"):
    if task_run != 'just_face':
        params_file = dataDir + "schlichting_Base_Params_TaskColorRecall.csv"
    else: 
        params_file = dataDir + "schlichting_Base_Params_TaskFaceRecall.csv"
        
    # Create a dictionary of Layer:{DICTIONARY of param values}
    Layer_Param_dict = defaultdict(dict)
    with open(params_file, 'r') as f:
        for line in f.readlines():
            if line[:6] == "Layer:": # get layer name
                layername = line.split(" ")[-1][:-1]
                
            if line[:5] == "Prjn:": # get layer name
                prjnname = line.split(" ")[-1][:-1]
                
            if "OscAmnt" in line: # get oscAmnt
                Layer_Param_dict[layername]["OscAmnt"] = float(line.split(" ")[-2])
                
            if "K_max" in line: # get K_max
                Layer_Param_dict[layername]["K_max"] = float(line.split(" ")[7])
                
            if "K_for_WTA" in line: # get K_max
                Layer_Param_dict[layername]["K_for_WTA"] = float(line.split(" ")[5])
                
            if "K_point" in line: # get K_max
                Layer_Param_dict[layername]["K_point"] = float(line.split(" ")[9])
                
            if "Target_diff" in line: # get K_max
                Layer_Param_dict[layername]["Target_diff"] = float(line.split(" ")[11])
                
            if "VmActThr" in line: # get K_max
                Layer_Param_dict[layername]["XX1_Gain"] = float(line.split(" ")[9])
                
            if "AvgGain" in line: # get K_max
                Layer_Param_dict[layername]["Clamp_Gain"] = float(line.split(" ")[22])
                
    # Create a dictionary of Param_values:{LIST of Layers}
    print("Layer_Param_dict", Layer_Param_dict)
    Param_Layer_dict = defaultdict(list)
    for layer, param_dict in Layer_Param_dict.items():
        Param_Layer_dict["Model"] = model_name
        Param_Layer_dict["Layer"].append(layer)
        for param_name, param_value in param_dict.items():
            if param_name == "K_max" and param_value == -1:
                param_value = "+inf"
            elif param_name == "Clamp_Gain" and (layer == "Hidden" or layer == "Output"):
                param_value = "---"
            Param_Layer_dict[param_name].append(param_value)
    Param_Layer_df = pd.DataFrame.from_dict(Param_Layer_dict)
    cols = ['Model', 'Layer', 'K_for_WTA', 'K_max', 'K_point', 'Target_diff', 'OscAmnt', 'XX1_Gain', 'Clamp_Gain']
    Param_Layer_df = Param_Layer_df[cols]
    Param_Layer_df.to_csv(dataDir + "schlichting_Layer_Params.csv", index=False)
    return Param_Layer_df
        

print('getting parameter values', flush=True)

parameter_values = get_parameter_values()
display(parameter_values)

getting parameter values
Layer_Param_dict defaultdict(<class 'dict'>, {'Category': {'XX1_Gain': 100.0, 'Clamp_Gain': 2.0, 'K_max': -1.0, 'K_for_WTA': 1.0, 'K_point': 0.75, 'Target_diff': 0.0, 'OscAmnt': 0.0}, 'Scene': {'XX1_Gain': 100.0, 'Clamp_Gain': 0.3, 'K_max': -1.0, 'K_for_WTA': 1.0, 'K_point': 0.95, 'Target_diff': 0.2, 'OscAmnt': 0.156}, 'Hidden': {'XX1_Gain': 100.0, 'Clamp_Gain': 0.2, 'K_max': 10.0, 'K_for_WTA': 6.0, 'K_point': 0.8, 'Target_diff': 0.02, 'OscAmnt': 0.0623}, 'Output': {'XX1_Gain': 100.0, 'Clamp_Gain': 1.0, 'K_max': -1.0, 'K_for_WTA': 1.0, 'K_point': 0.75, 'Target_diff': 0.03, 'OscAmnt': 0.03}})


,Model,Layer,K_for_WTA,K_max,K_point,Target_diff,OscAmnt,XX1_Gain,Clamp_Gain
0,Chanales,Category,1.0,-1.0,0.75,0.00,0.0000,100.0,2.0
1,Chanales,Scene,1.0,-1.0,0.95,0.20,0.1560,100.0,0.3
2,Chanales,Hidden,6.0,10.0,0.80,0.02,0.0623,100.0,0.2
3,Chanales,Output,1.0,-1.0,0.75,0.03,0.0300,100.0,1.0


In [18]:
def get_parameter_values(model_name = "Chanales"):
    if task_run != 'just_face':
        params_file = dataDir + "schlichting_Base_Params_TaskColorRecall.csv"
    else: 
        params_file = dataDir + "schlichting_Base_Params_TaskFaceRecall.csv"
        
    # Create a dictionary of Layer:{DICTIONARY of param values}
    Prjn_Param_dict = defaultdict(dict)
    with open(params_file, 'r') as f:
        for line in f.readlines():
            if line[:6] == "Layer:": # get layer name
                layername = line.split(" ")[-1][:-1]
                
            if line[:5] == "Prjn:": # get layer name
                prjnname = line.split(" ")[-1][:-1]
                
            if "DThr_NMPH" in line: # get DThr_NMPH
                idx = line.split(" ").index("DThr_NMPH:")
                Prjn_Param_dict[prjnname]["DThr_NMPH"] = float(line.split(" ")[idx + 1])
                
            if "DRev_NMPH" in line: # get DRev_NMPH
                idx = line.split(" ").index("DRev_NMPH:")
                Prjn_Param_dict[prjnname]["DRev_NMPH"] = float(line.split(" ")[idx + 1])
                
            if "DRevMag_NMPH" in line: # get DRevMag_NMPH
                idx = line.split(" ").index("DRevMag_NMPH:")
                Prjn_Param_dict[prjnname]["DRevMag_NMPH"] = float(line.split(" ")[idx + 1])

            if "ThrP_NMPH" in line: # get ThrP_NMPH
                idx = line.split(" ").index("ThrP_NMPH:")
                Prjn_Param_dict[prjnname]["ThrP_NMPH"] = float(line.split(" ")[idx + 1])
                
            if "DMaxMag_NMPH" in line: # get DMaxMag_NMPH
                idx = line.split(" ").index("DMaxMag_NMPH:")
                Prjn_Param_dict[prjnname]["DMaxMag_NMPH"] = float(line.split(" ")[idx + 1])
                
            if "Abs:" in line: # get wtscale
                Prjn_Param_dict[prjnname]["WtScale_Abs (forwards / backwards)"] = float(line.split(" ")[3])
                Prjn_Param_dict[prjnname]["WtScale_Rel (forwards / backwards)"] = float(line.split(" ")[5])
                
            if "InitStrategy" in line: # get wtscale
                mean = float(line.split(" ")[5])
                var = float(line.split(" ")[7])
                Prjn_Param_dict[prjnname]["Wt Range"] = f"{mean - var} - {mean + var}"
                
    # Create a dictionary of Param_values:{LIST of Layers}
    print("Prjn_Param_dict", Prjn_Param_dict)
    Layer_Order = {"Category": 0, "Scene": 1, "Hidden": 2, "Output": 3}
    Param_Prjn_dict = defaultdict(list)
    for prjn, param_dict in Prjn_Param_dict.items():
        #print("prjn", prjn.split("To"))
        send, recv = prjn.split("To")
        if Layer_Order[send] > Layer_Order[recv]:
            continue
        Param_Prjn_dict["Model"] = model_name
        Param_Prjn_dict["Projection"].append(prjn)
        for param_name, param_value in param_dict.items():
            if "(forwards / backwards)" in param_name:
                reverse_prjn_name = f"{recv}To{send}"
                param_value = str(param_value) + f"/{Prjn_Param_dict[reverse_prjn_name][param_name]}"
            Param_Prjn_dict[param_name].append(param_value)
    Param_Prjn_df = pd.DataFrame.from_dict(Param_Prjn_dict)
    cols = Param_Prjn_df.columns.tolist() 
    Param_Prjn_df = Param_Prjn_df[cols]
    Param_Prjn_df.to_csv(dataDir + "schlichting_Prjn_Params.csv", index=False)
    return Param_Prjn_df

print('getting parameter values', flush=True)

parameter_values = get_parameter_values() 
display(parameter_values)

getting parameter values
Prjn_Param_dict defaultdict(<class 'dict'>, {'HiddenToCategory': {'Wt Range': '0.01 - 0.03', 'WtScale_Abs (forwards / backwards)': 0.1, 'WtScale_Rel (forwards / backwards)': 1.0, 'DThr_NMPH': 0.2, 'DRev_NMPH': 0.3, 'DRevMag_NMPH': -0.1, 'ThrP_NMPH': 0.46, 'DMaxMag_NMPH': 0.06}, 'HiddenToScene': {'Wt Range': '0.45 - 0.55', 'WtScale_Abs (forwards / backwards)': 0.2, 'WtScale_Rel (forwards / backwards)': 1.0, 'DThr_NMPH': 0.11, 'DRev_NMPH': 0.23, 'DRevMag_NMPH': -1.5, 'ThrP_NMPH': 0.4, 'DMaxMag_NMPH': 1.0}, 'SceneToHidden': {'Wt Range': '0.45 - 0.55', 'WtScale_Abs (forwards / backwards)': 0.3, 'WtScale_Rel (forwards / backwards)': 1.0, 'DThr_NMPH': 0.11, 'DRev_NMPH': 0.23, 'DRevMag_NMPH': -1.5, 'ThrP_NMPH': 0.4, 'DMaxMag_NMPH': 1.0}, 'CategoryToHidden': {'Wt Range': '0.01 - 0.03', 'WtScale_Abs (forwards / backwards)': 0.2, 'WtScale_Rel (forwards / backwards)': 1.0, 'DThr_NMPH': 0.2, 'DRev_NMPH': 0.3, 'DRevMag_NMPH': -0.1, 'ThrP_NMPH': 0.46, 'DMaxMag_NMPH': 0.06}, 

,Model,Projection,Wt Range,WtScale_Abs (forwards / backwards),WtScale_Rel (forwards / backwards),DThr_NMPH,DRev_NMPH,DRevMag_NMPH,ThrP_NMPH,DMaxMag_NMPH
0,Chanales,SceneToHidden,0.45 - 0.55,0.3/0.2,1.0/1.0,0.11,0.23,-1.50,0.40,1.00
1,Chanales,CategoryToHidden,0.01 - 0.03,0.2/0.1,1.0/1.0,0.20,0.30,-0.10,0.46,0.06
2,Chanales,HiddenToHidden,0.45 - 0.55,1.9/1.9,1.0/1.0,0.11,0.23,-1.50,0.40,1.00
3,Chanales,HiddenToOutput,0.01 - 0.03,1.2/1.7,1.0/1.0,0.11,0.23,-0.01,0.40,0.50
